In [1]:
# Asthetics https://www.kaggle.com/manabendrarout/tabular-data-preparation-basic-eda-and-baseline
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Basic
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
import os
import random
from tqdm.autonotebook import tqdm
import string
import re
from functools import partial
from ipywidgets import IntProgress

In [2]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [3]:
train_df.head(3)

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


In [4]:
def text_cleaning(text):
    text = re.sub(r'[^A-Za-z0-9.!?'"'"'()\[\]]+', ' ', text)
    text = re.sub("'", '', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('[.]{2,}', '.', text)
    text = re.sub(r'\. \.', '.', text)
    text = re.sub(r' \.', '.', text)    
    
    return text

In [5]:
def read_append_return(filename, train_files_path=train_files_path, output='text'):
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f: #encoding='utf-8'
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            s = text_cleaning(data.get('text'))
            if len(s) > 200000:
                #s = data.get('text')
                #print(data.get('text'))
                l = s.split()
                n = 100000
                texto = [" ".join(l[x:x+n]) for x in range(0, len(l), n)]   
                contents.extend(texto)
            else:
                contents.append(s)
                #print(contents1)
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    #contents = contents1.extend(contents2)
    #print(combined)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    elif output == 'comb':
        return contents
    else:
        #print("FEDE")
        return all_data

In [6]:
#tqdm.pandas()
#train_df['text_all'] = train_df['Id'].progress_apply(partial(read_append_return, output='all'))

In [7]:
#tqdm.pandas()
#train_df['text_all'] = train_df['text_all'].progress_apply(text_cleaning)

In [8]:
tqdm.pandas()
sample_sub['text_all'] = sample_sub['Id'].progress_apply(partial(read_append_return, train_files_path=test_files_path, output='all'))

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
tqdm.pandas()
sample_sub['text_all'] = sample_sub['text_all'].progress_apply(text_cleaning)

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
sample_sub.head()

,Id,PredictionString,text_all
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Abstract. Cognitive deficits and reduced educa...
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,Introduction. This report describes how the ed...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,INTRODUCTION. Cape Hatteras National Seashore ...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,Introduction. A significant body of research h...


In [11]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [12]:
import spacy
from spacy.language import Language
from typing import Tuple, List, Optional, Set, Dict
from collections import defaultdict
from spacy.tokens import Span, Doc
from spacy.matcher import Matcher
from spacy.pipeline import EntityRuler

In [13]:
def find_abbreviation(
    long_form_candidate: Span, short_form_candidate: Span
) -> Tuple[Span, Optional[Span]]:
    """
    Implements the abbreviation detection algorithm in "A simple algorithm
    for identifying abbreviation definitions in biomedical text.", (Schwartz & Hearst, 2003).

    The algorithm works by enumerating the characters in the short form of the abbreviation,
    checking that they can be matched against characters in a candidate text for the long form
    in order, as well as requiring that the first letter of the abbreviated form matches the
    _beginning_ letter of a word.

    Parameters
    ----------
    long_form_candidate: Span, required.
        The spaCy span for the long form candidate of the definition.
    short_form_candidate: Span, required.
        The spaCy span for the abbreviation candidate.

    Returns
    -------
    A Tuple[Span, Optional[Span]], representing the short form abbreviation and the
    span corresponding to the long form expansion, or None if a match is not found.
    """
    long_form = " ".join([x.text for x in long_form_candidate])
    short_form = " ".join([x.text for x in short_form_candidate])

    long_index = len(long_form) - 1
    short_index = len(short_form) - 1

    while short_index >= 0:
        current_char = short_form[short_index].lower()
        # We don't check non alpha-numeric characters.
        if not current_char.isalnum():
            short_index -= 1
            continue

            # Does the character match at this position? ...
        while (
            (long_index >= 0 and long_form[long_index].lower() != current_char)
            or
            # .... or if we are checking the first character of the abbreviation, we enforce
            # to be the _starting_ character of a span.
            (
                short_index == 0
                and long_index > 0
                and long_form[long_index - 1].isalnum()
            )
        ):
            long_index -= 1

        if long_index < 0:
            return short_form_candidate, None

        long_index -= 1
        short_index -= 1

    # The last subtraction will either take us on to a whitespace character, or
    # off the front of the string (i.e. long_index == -1). Either way, we want to add
    # one to get back to the start character of the long form
    long_index += 1

    # Now we know the character index of the start of the character span,
    # here we just translate that to the first token beginning after that
    # value, so we can return a spaCy span instead.
    word_lengths = 0
    starting_index = None
    for i, word in enumerate(long_form_candidate):
        # need to add 1 for the space characters
        word_lengths += len(word.text_with_ws)
        if word_lengths > long_index:
            starting_index = i
            break

    return short_form_candidate, long_form_candidate[starting_index:]

def filter_matches(
    matcher_output: List[Tuple[int, int, int]], doc: Doc
) -> List[Tuple[Span, Span]]:
    # Filter into two cases:
    # 1. <Short Form> ( <Long Form> )
    # 2. <Long Form> (<Short Form>) [this case is most common].
    candidates = []
    for match in matcher_output:
        start = match[1]
        end = match[2]
        # Ignore spans with more than 8 words in them, and spans at the start of the doc
        if end - start > 8 or start == 1:
            continue
        if end - start > 3:
            # Long form is inside the parens.
            # Take one word before.
            short_form_candidate = doc[start - 2 : start - 1]
            long_form_candidate = doc[start:end]
        else:
            # Normal case.
            # Short form is inside the parens.
            short_form_candidate = doc[start:end]

            # Sum character lengths of contents of parens.
            abbreviation_length = sum([len(x) for x in short_form_candidate])
            max_words = min(abbreviation_length + 5, abbreviation_length * 2)
            # Look up to max_words backwards
            long_form_candidate = doc[max(start - max_words - 1, 0) : start - 1]

        # add candidate to candidates if candidates pass filters
        if short_form_filter(short_form_candidate):
            candidates.append((long_form_candidate, short_form_candidate))

    return candidates

def short_form_filter(span: Span) -> bool:
    # All words are between length 2 and 10
    if not all([2 <= len(x) < 10 for x in span]):
        return False

    # At least 50% of the short form should be alpha
    if (sum([c.isalpha() for c in span.text]) / len(span.text)) < 0.5:
        return False

    # The first character of the short form should be alpha
    if not span.text[0].isalpha():
        return False
    return True

class AbbreviationDetector:
    """
    Detects abbreviations using the algorithm in "A simple algorithm for identifying
    abbreviation definitions in biomedical text.", (Schwartz & Hearst, 2003).

    This class sets the `._.abbreviations` attribute on spaCy Doc.

    The abbreviations attribute is a `List[Span]` where each Span has the `Span._.long_form`
    attribute set to the long form definition of the abbreviation.

    Note that this class does not replace the spans, or merge them.
    """

    def __init__(self, nlp) -> None:
        Doc.set_extension("abbreviations", default=[], force=True)
        Span.set_extension("long_form", default=None, force=True)

        self.matcher = Matcher(nlp.vocab)
        self.matcher.add(
            "parenthesis", None, [{"ORTH": "("}, {"OP": "+"}, {"ORTH": ")"}]
        )
        self.global_matcher = Matcher(nlp.vocab)

    def find(self, span: Span, doc: Doc) -> Tuple[Span, Set[Span]]:
        """
        Functional version of calling the matcher for a single span.
        This method is helpful if you already have an abbreviation which
        you want to find a definition for.
        """
        dummy_matches = [(-1, int(span.start), int(span.end))]
        filtered = filter_matches(dummy_matches, doc)
        abbreviations = self.find_matches_for(filtered, doc)

        if not abbreviations:
            return span, set()
        else:
            return abbreviations[0]
    def __call__(self, doc: Doc) -> Doc:
        matches = self.matcher(doc)
        matches_no_brackets = [(x[0], x[1] + 1, x[2] - 1) for x in matches]
        filtered = filter_matches(matches_no_brackets, doc)
        occurences = self.find_matches_for(filtered, doc)

        for (long_form, short_forms) in occurences:
            for short in short_forms:
                short._.long_form = long_form
                doc._.abbreviations.append(short)
        return doc

    def find_matches_for(
        self, filtered: List[Tuple[Span, Span]], doc: Doc
    ) -> List[Tuple[Span, Set[Span]]]:
        rules = {}
        all_occurences: Dict[Span, Set[Span]] = defaultdict(set)
        already_seen_long: Set[str] = set()
        already_seen_short: Set[str] = set()
        for (long_candidate, short_candidate) in filtered:
            short, long = find_abbreviation(long_candidate, short_candidate)
            # We need the long and short form definitions to be unique, because we need
            # to store them so we can look them up later. This is a bit of a
            # pathalogical case also, as it would mean an abbreviation had been
            # defined twice in a document. There's not much we can do about this,
            # but at least the case which is discarded will be picked up below by
            # the global matcher. So it's likely that things will work out ok most of the time.
            new_long = long.string not in already_seen_long if long else False
            new_short = short.string not in already_seen_short
            if long is not None and new_long and new_short:
                already_seen_long.add(long.string)
                already_seen_short.add(short.string)
                all_occurences[long].add(short)
                rules[long.string] = long
                # Add a rule to a matcher to find exactly this substring.
                self.global_matcher.add(
                    long.string, None, [{"ORTH": x.text} for x in short]
                )
        to_remove = set()
        global_matches = self.global_matcher(doc)
        for match, start, end in global_matches:
            string_key = self.global_matcher.vocab.strings[match]
            to_remove.add(string_key)
            all_occurences[rules[string_key]].add(doc[start:end])
        for key in to_remove:
            # Clean up the global matcher.
            self.global_matcher.remove(key)
        return list((k, v) for k, v in all_occurences.items())

In [14]:
nlp = spacy.load("../input/nerlast716/NER-LAST-VEC-716-RULER", disable = ['tagger', 'ner'])

In [15]:
# Add the abbreviation pipe to the spacy pipeline.
abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)
Language.factories['AbbreviationDetector'] = lambda nlp, **cfg: AbbreviationDetector(nlp, **cfg)
nlp.max_length = 15000000
print(nlp.pipe_names)

['parser', 'entity_ruler', 'AbbreviationDetector']


In [16]:
int_org_df = pd.read_csv('../input/last-int-org/International_ORG_sin_duplicados.csv')
list3 = int_org_df['Text_Org'].tolist()

to_vec = pd.read_csv('../input/last-for-vectors/cleanedLabel_toVec_Diego_for_vectors.csv')
list2 = [text_cleaning(x).lower().rstrip() for x in to_vec['0'].unique()]

list2 = set(list2)
len(list2)

1039

In [17]:
nlp2 = spacy.load('../input/ensemble-textcat/Ensemble_30000_textcat')
nlp3 = spacy.load('../input/ner-last-1100/NER-LAST-VEC-1100-RULER', disable = ['tagger', 'parser'])
nlp4 = spacy.load('../input/dataset-vs-org/Ensemble_3000_dataset_vs_ORG')

print(nlp4.pipe_names)

['textcat']


In [18]:
#Vectores
list2SpacyDocs = [nlp(x) for x in list2]

In [19]:
#train_df_unique = train_df.drop_duplicates(subset=['Id'])
#print (len(train_df_unique['Id']))

In [20]:
#Abvr-NER-Sentences Combino 2 modelos nlp
id_list = []
lables_list = []
threshold = 0.90
threshold_data = 0.95
ne = 0
maxlen=1000000

for index, row in tqdm(sample_sub.iterrows()):
    llabels = []
    papertxt = row['text_all']
    row_id = row['Id']
    nlp_labels = []
    docs = []
    if len(papertxt) > maxlen:
        i = round(len(papertxt)/maxlen)-1
        while i >= 0:
            docs.append(papertxt[0:maxlen-1])
            papertxt = papertxt[maxlen:]
            i = i - 1
    else:
        docs.append(papertxt)
        
    abvr_list = []
    if docs:
        for text in docs:
            doc = nlp(text)
            for abrv in doc._.abbreviations:
                if any(abrv.text in s for s in abvr_list):
                    pass
                else:
                    if abrv._.long_form.text not in list3 and abrv.text != "AD" and not abrv.text.islower() and not abrv._.long_form.text.islower(): 
                        #print(abrv.text)
                        long_data = abrv._.long_form.text + ", " + abrv.text
                        if abvr_list:
                            if any(abrv.text in s for s in abvr_list):
                                pass
                            else:
                                abvr_list.append(long_data)                           
                        else:
                            abvr_list.append(long_data)
                            
            #print(abvr_list)
            #"""
            en = 0
            for ent in doc.ents:
                en += 1            
                if ent.label_ == "RULDATA":                    
                    #print("In list: ", ent.text)
                    if any(ent.text.lower() in s.lower() for s in abvr_list):
                        for words in abvr_list:
                            palabras = words.split(", ")
                            #print(palabras[0])
                            abrv_word = palabras[-1]
                            #print(abrv_word)
                            if abrv_word in ent.text or palabras[0] in ent.text:
                                #print("From Ruler: ", words)
                                nlp_labels.append(words)
                                nlp_labels.append(abrv_word)                             
                    else:                        
                        if len(ent) > 2:
                            long_ent = ent.text
                            long_ent_find = re.findall('\(([^)]+)', long_ent)
                            if long_ent_find:
                                abrv_ent = re.search('\(([^)]+)', long_ent).group(1)                        
                                nlp_labels.append(long_ent)
                                nlp_labels.append(abrv_ent)
                            else:
                                nlp_labels.append(long_ent)
            #"""
            #print(abvr_list)
            #Vectores
            labels_vec_set = set(nlp_labels)
            #print(labels_vec_set)
            label_vec = [nlp(x.lower()) for x in labels_vec_set]
            np = 0
            for sent in doc.sents:
                """
                slen = len(sent.text.split())
                if slen >= 7 and slen <= 100:
                    strsent = str(sent.text)
                    strsent = strsent.strip()
                    doccat = nlp2(strsent)
                    if doccat.cats['SI'] > threshold:
                """
                np += 1
                #docent = nlp3(strsent)
                docent = nlp3(sent.text)
                for ent in docent.ents:                            
                    if any(ent.text.lower() in s.lower() for s in abvr_list) and not any(ent.text.lower() in s.lower() for s in labels_vec_set): 
                        #print("In list: ", ent.text)
                        for s in abvr_list:                                    
                            if ent.text.lower() in s.lower():                                            
                                palabras = s.split(", ")
                                data_vs_org = nlp4(palabras[0].lower())                                
                                #print("From ents: ", palabras)
                                if data_vs_org.cats['SI'] > threshold_data:                                            
                                    nlp_labels.append(palabras[0])
                                    nlp_labels.append(palabras[-1]) 

                    else:
                        paso = False
                        text1 = nlp(ent.text.lower())                                
                        data_vs_org = nlp4(ent.text)                                
                        #print(data_vs_org.cats['SI'])
                        if data_vs_org.cats['SI'] > threshold_data:
                            #nlp_labels.append(ent.text)
                            paso = True                                    
                            for vec in label_vec:
                                #print("Lista ruler: ", vec)
                                similarity2 = text1.similarity(vec)
                                #print(similarity2)
                                #print("Lista ner: ", text1)
                                #print("........")                                        
                                if similarity2 > 0.90:
                                    #print(ent.text)
                                    paso = False
                        if paso:
                            #print(ent.text)
                            nlp_labels.append(ent.text)                
            
    cleaned_labels = [clean_text(x) for x in nlp_labels]
    #cleaned_labels =[]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(str(row_id))
    #print("LABELS")
    print(cleaned_labels)
    print('Sentences classified as positive:', np)
    print('Rul-Ents classified as positive:', en)

0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:111: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


{'alzheimers disease neuroimaging initiative adni', 'charge', 'genomes project', 'fhs', 'cardiovascular health study chs', 'adni', 'framingham heart study second generation cohort fhs', 'birth cohort study hbcs', 'cohorts for heart and aging research in genomic epidemiology charge', 'chs', 'ping', 'pediatric imaging neurocognition and genetics ping'}
Sentences classified as positive: 218
Rul-Ents classified as positive: 12
{'timss', 'sass', 'pisa', 'common core of data ccd', 'ccd', 'cps', 'progress in international reading literacy study pirls', 'integrated postsecondary education data system ipeds', 'trends in international mathematics and science study timss', 'pirls', 'schools and staffing survey sass', 'national accounts database oecd 2008b', 'current population survey cps', 'ipeds', 'program for international student assessment pisa'}
Sentences classified as positive: 1461
Rul-Ents classified as positive: 131
{'geological survey usgs', 'coastal erosion study', 'gis', 'noaa storm s

In [21]:
"""
#Abvr-NER-Sentences Combino 2 modelos nlp
id_list = []
lables_list = []
threshold = 0.70
ne = 0
maxlen=1000000

for index, row in tqdm(sample_sub.iterrows()):
    llabels = []
    papertxt = row['text_all']
    row_id = row['Id']
    nlp_labels = []
    docs = []
    if len(papertxt) > maxlen:
        i = round(len(papertxt)/maxlen)-1
        while i >= 0:
            docs.append(papertxt[0:maxlen-1])
            papertxt = papertxt[maxlen:]
            i = i - 1
    else:
        docs.append(papertxt)
        
    abvr_list = []
    if docs:
        for text in docs:
            doc = nlp(text)
            for abrv in doc._.abbreviations:
                if any(abrv.text in s for s in abvr_list):
                    pass
                else:
                    if abrv._.long_form.text not in list3 and abrv.text != "AD" and not abrv.text.islower() and not abrv._.long_form.text.islower():
                        #print(abrv.text)
                        long_data = abrv._.long_form.text + ", " + abrv.text
                        if abvr_list:
                            if any(abrv.text in s for s in abvr_list):
                                pass
                            else:
                                abvr_list.append(long_data)                           
                        else:
                            abvr_list.append(long_data)
                            
            #print(abvr_list)
            en = 0
            for ent in doc.ents:
                en += 1
                if ent.label_ == "RULDATA":                    
                    #print("In list: ", ent.text)
                    if any(ent.text.lower() in s.lower() for s in abvr_list):
                        for words in abvr_list:
                            palabras = words.split(", ")
                            #print(palabras[0])
                            abrv_word = palabras[-1]
                            #print(abrv_word)
                            if abrv_word in ent.text or palabras[0] in ent.text:
                                #print(ent.text)
                                nlp_labels.append(words)
                                nlp_labels.append(abrv_word)                             
                    else:                        
                        if len(ent) > 2:
                            long_ent = ent.text
                            long_ent_find = re.findall('\(([^)]+)', long_ent)
                            if long_ent_find:
                                abrv_ent = re.search('\(([^)]+)', long_ent).group(1)                        
                                nlp_labels.append(long_ent)
                                nlp_labels.append(abrv_ent)
                            else:
                                nlp_labels.append(long_ent)
                                
            #print(abvr_list)
            #Vectores
            labels_vec_set = set(nlp_labels)
            #print(labels_vec_set)
            label_vec = [nlp(x.lower()) for x in labels_vec_set]
            np = 0
            for sent in doc.sents:
                slen = len(sent.text.split())
                if slen >= 7 and slen <= 100:
                    strsent = str(sent.text)
                    strsent = strsent.strip()
                    doccat = nlp2(strsent)
                    if doccat.cats['SI'] > threshold:
                        np += 1
                        docent = nlp3(strsent)
                        for ent in docent.ents:                            
                            if any(ent.text.lower() in s.lower() for s in abvr_list) and not any(ent.text.lower() in s.lower() for s in labels_vec_set):
                                #print("In list: ", ent.text)
                                for s in abvr_list:                                    
                                    if ent.text.lower() in s.lower():                                            
                                        palabras = s.split(", ")
                                        text1 = nlp(palabras[0].lower())                                
                                        #print(text1)
                                        for y in list2SpacyDocs:
                                            similarity = text1.similarity(y)
                                            if similarity > 0.80:                                                    
                                                #print(palabras[0], ent.text)
                                                nlp_labels.append(palabras[0])
                                                nlp_labels.append(palabras[-1])
                                                break
                                            #if similarity > 0.80: 
                                            #    print("Lista Abv-ruler: ", text1)
                                            #    print(similarity)
                                            #    print("Lista ner: ", y)
                                            #    print("........")  
                                            
                            else:
                                paso = False
                                text1 = nlp(ent.text.lower())                                
                                for y in list2SpacyDocs:                                    
                                    #if len(text1) == len(y):
                                    #print(ent.text)                            
                                    #print(y)
                                    similarity = text1.similarity(y)
                                    if similarity > 0.90:                                    
                                        paso = True
                                        for vec in label_vec:
                                            #print("Lista ruler: ", vec)
                                            similarity2 = text1.similarity(vec)
                                            #print(similarity2)
                                            #print("Lista ner: ", text1)
                                            #print("........")                                        
                                            if similarity2 > 0.94:
                                                paso = False
                                if paso:
                                    #print(ent.text)
                                    nlp_labels.append(ent.text)                
            
    cleaned_labels = [clean_text(x) for x in nlp_labels]
    #cleaned_labels =[]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(str(row_id))
    #print("LABELS")
    print(cleaned_labels)
    print('Sentences classified as positive:', np)
    print('Rul-Ents classified as positive:', en)
"""
print("nada... Score 54%")

nada... Score 54%


In [22]:
"""
#Abvr-NER-Sentences
id_list = []
lables_list = []
threshold = 0.70
ne = 0
maxlen=1000000

for index, row in tqdm(sample_sub.iterrows()):
    llabels = []
    papertxt = row['text_all']
    row_id = row['Id']
    nlp_labels = []
    docs = []
    if len(papertxt) > maxlen:
        i = round(len(papertxt)/maxlen)-1
        while i >= 0:
            docs.append(papertxt[0:maxlen-1])
            papertxt = papertxt[maxlen:]
            i = i - 1
    else:
        docs.append(papertxt)
        
    abvr_list = []
    if docs:
        for text in docs:
            doc = nlp(text)
            for abrv in doc._.abbreviations:
                if any(abrv.text in s for s in abvr_list):
                    pass
                else:
                    if abrv._.long_form.text not in list3 and abrv.text != "AD" and not abrv.text.islower() and not abrv._.long_form.text.islower():
                        #print(abrv.text)
                        long_data = abrv._.long_form.text + ", " + abrv.text
                        if abvr_list:
                            if any(abrv.text in s for s in abvr_list):
                                pass
                            else:
                                abvr_list.append(long_data)                           
                        else:
                            abvr_list.append(long_data)
                            
            #print(abvr_list)
            np = 0
            
            for sent in doc.sents:
                slen = len(ent.sent.text.split())
                if slen >= 7 and slen <= 100:
                    strsent = ent.sent.text
                    strsent = strsent.strip()
                    doccat = nlp2(strsent)
                    if doccat.cats['SI'] > threshold:
                        np += 1
            
            en = 0
            for ent in doc.ents:
                en += 1
                if ent.label_ == "RULDATA":                    
                    #print("In list: ", ent.text)
                    if any(ent.text.lower() in s.lower() for s in abvr_list):
                        for words in abvr_list:
                            palabras = words.split(", ")
                            #print(palabras[0])
                            abrv_word = palabras[-1]
                            #print(abrv_word)
                            if abrv_word in ent.text or palabras[0] in ent.text:
                                #print(ent.text)
                                nlp_labels.append(words)
                                nlp_labels.append(abrv_word)                             
                    else:                        
                        if len(ent) > 2:
                            long_ent = ent.text
                            long_ent_find = re.findall('\(([^)]+)', long_ent)
                            if long_ent_find:
                                abrv_ent = re.search('\(([^)]+)', long_ent).group(1)                        
                                nlp_labels.append(long_ent)
                                nlp_labels.append(abrv_ent)
                            else:
                                nlp_labels.append(long_ent)
                                
                else:                    
                    slen = len(ent.sent.text.split())
                    if slen >= 7 and slen <= 100:
                        strsent = ent.sent.text
                        strsent = strsent.strip()
                        doccat = nlp2(strsent)
                        if doccat.cats['SI'] > threshold:
                            #print("In list: ", ent.text)
                            np += 1
                            if any(ent.text.lower() in s.lower() for s in abvr_list):
                                #print("In list: ", ent.text)
                                for s in abvr_list:
                                    if ent.text.lower() in s.lower():                                            
                                        palabras = s.split(", ")
                                        text1 = nlp(palabras[0].lower())                                
                                        #print(text1)
                                        for y in list2SpacyDocs:                                    
                                            #if len(text1) == len(y):
                                            #print(ent.text)                            
                                            #print(y)
                                            similarity = text1.similarity(y)
                                            if similarity > 0.85:                                                    
                                                #print(palabras[0], ent.text)
                                                nlp_labels.append(palabras[0])
                                                nlp_labels.append(ent.text)
                                                #print(y)
                                                #print(similarity)
                                                #print(text1)
                                                break
                            else:    
                                text1 = nlp(ent.text.lower())                                
                                for y in list2SpacyDocs:                                    
                                    #if len(text1) == len(y):
                                    #print(ent.text)                            
                                    #print(y)
                                    similarity = text1.similarity(y)
                                    if similarity > 0.90:                                        
                                        #print(ent.text)
                                        nlp_labels.append(ent.text)
                                        #print(y)
                                        #print(similarity)
                                        #print(text1)
                                        break                
                         
            
    cleaned_labels = [clean_text(x) for x in nlp_labels]
    #cleaned_labels =[]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(str(row_id))
    #print("LABELS")
    print(cleaned_labels)
    print('Sentences classified as positive:', np)
    print('Ents classified as positive:', en)
"""
print("nada...")

nada...


In [23]:
"""
#Abvr-NER-Sentences
id_list = []
lables_list = []
threshold = 0.80
ne = 0
maxlen=1000000

for index, row in tqdm(train_df[12000:12050].iterrows()):
    llabels = []
    papertxt = row['text_all']
    row_id = row['Id']
    nlp_labels = []
    docs = []
    if len(papertxt) > maxlen:
        i = round(len(papertxt)/maxlen)-1
        while i >= 0:
            docs.append(papertxt[0:maxlen-1])
            papertxt = papertxt[maxlen:]
            i = i - 1
    else:
        docs.append(papertxt)
        
    abvr_list = []
    if docs:
        for text in docs:
            doc = nlp(text)
            for abrv in doc._.abbreviations:
                if any(abrv.text in s for s in abvr_list):
                    pass
                else:
                    if abrv._.long_form.text not in list3 and abrv.text != "AD" and not abrv.text.islower() and not abrv._.long_form.text.islower():
                        #print(abrv.text)
                        long_data = abrv._.long_form.text + ", " + abrv.text
                        if abvr_list:
                            if any(abrv.text in s for s in abvr_list):
                                pass
                            else:
                                abvr_list.append(long_data)                           
                        else:
                            abvr_list.append(long_data)
                            
            #print(abvr_list)
            np = 0
            for sent in doc.sents:
                slen = len(sent.text.split())
                if slen >= 7 and slen <= 100:
                    strsent = str(sent.text)
                    strsent = strsent.strip()
                    doccat = nlp2(strsent)
                    if doccat.cats['SI'] > threshold:
                        np += 1
                        docent = nlp(strsent)
                        for ent in docent.ents:
                            #print(doccat.text, "(" ,ent.text, ")")
                            if any(ent.text in s for s in abvr_list):
                                for words in abvr_list:
                                    palabras = words.split(", ")
                                    #print(palabras[0])
                                    abrv_word = palabras[-1]
                                    #print(abrv_word)
                                    if abrv_word in ent.text or palabras[0] in ent.text:
                                        #print(ent.text)
                                        nlp_labels.append(words)
                                        nlp_labels.append(abrv_word)
                            else:
                                #print(doccat.text, "(" ,ent.text, ")")
                                if len(ent) > 2:
                                    long_ent = ent.text
                                    long_ent_find = re.findall('\(([^)]+)', long_ent)
                                    if long_ent_find:
                                        abrv_ent = re.search('\(([^)]+)', long_ent).group(1)                        
                                        nlp_labels.append(long_ent)
                                        nlp_labels.append(abrv_ent)
                                    else:
                                        nlp_labels.append(long_ent)            
            
    cleaned_labels = [clean_text(x) for x in nlp_labels]
    #cleaned_labels =[]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(str(row_id))
    #print("LABELS")
    print(cleaned_labels)
    print('Sentences classified as positive:', np)
"""
print("nada...")

nada...


In [24]:
submission = pd.DataFrame()
submission['Id'] = id_list
submission['PredictionString'] = lables_list

In [25]:
submission.to_csv('submission.csv', index=False)
!head submission.csv

Id,PredictionString
2100032a-7c33-4bff-97ef-690822c43466,alzheimers disease neuroimaging initiative adni|charge|genomes project|fhs|cardiovascular health study chs|adni|framingham heart study second generation cohort fhs|birth cohort study hbcs|cohorts for heart and aging research in genomic epidemiology charge|chs|ping|pediatric imaging neurocognition and genetics ping
2f392438-e215-4169-bebf-21ac4ff253e1,timss|sass|pisa|common core of data ccd|ccd|cps|progress in international reading literacy study pirls|integrated postsecondary education data system ipeds|trends in international mathematics and science study timss|pirls|schools and staffing survey sass|national accounts database oecd 2008b|current population survey cps|ipeds|program for international student assessment pisa
3f316b38-1a24-45a9-8d8c-4e05a42257c6,geological survey usgs|coastal erosion study|gis|noaa storm surge inundation|maximum envelope of water meow|geographic information systems gis|national hydrography dataset|se